<a href="https://colab.research.google.com/github/andrewc23/BFS_Search/blob/main/Distributed_BFS_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Distributed Implementation of Breadth First Search Using Apache Spark and SQL

Function takes in "G" (an undirected graph in the form of a Spark Dataframe), "origins" (a Python list of dictionaries), and max-depth (a non-negative integer). 

Function returns a pandas dataframe containing nodes and distances from the origin. 

In [ ]:

def spark_bfs(G, origins, max_depth):
  
  #initialize a view for the review_graph_sdf that was given 
  G.createOrReplaceTempView('review_graph')

  #make the undirected graph
  flipped_graph_sdf=spark.sql('''SELECT to_node, from_node FROM review_graph''')
  
  flipped_graph_sdf.createOrReplaceTempView('flipped_graph')
  
  #make the undirected graph
  undirected_graph_sdf=spark.sql('''SELECT to_node AS node_1, from_node as node_2 FROM flipped_graph UNION ALL SELECT from_node, to_node FROM review_graph ''')

  #create view of the undirected_graph_sdf
  undirected_graph_sdf.createOrReplaceTempView('undirected_graph')


  #create origins spark df
  schema_origin = StructType([
            StructField("node", StringType(), True)
        ])
  origins_sdf = spark.createDataFrame(origins, schema_origin)#create the origins sdf 
  #create the view of the origins sdf
  origins_sdf.createOrReplaceTempView('origins_df')


  #initiate the frontier variable
  frontier_sdf=spark.sql('''SELECT  origins_df.node AS frontier FROM origins_df ''')

  #create the view for the frontier
  frontier_sdf.createOrReplaceTempView('frontier_df')# only thing in this at this point should be the origins
  
  #frontier_sdf.show()

  #created the visited sdf 
  visited_sdf=spark.sql('''SELECT frontier_df.frontier as node FROM frontier_df''')

  #create the view for the visited sdf
  visited_sdf.createOrReplaceTempView('visited_df')

  distance = 0
  depth =0
  #add the distance to the visited 
  visited_sdf=visited_sdf.withColumn("distance",F.lit(distance))

  #create the view for the visited sdf
  visited_sdf.createOrReplaceTempView('visited_df')

  #visited_sdf.show()

  #loop to update the vistied and frontier dfs
  while depth < max_depth:

    #first update the frontier
     frontier_sdf=spark.sql('''SELECT undirected_graph.node_2 as Frontier  FROM frontier_df INNER JOIN undirected_graph ON undirected_graph.node_1=frontier_df.frontier''')

     frontier_sdf.createOrReplaceTempView('frontier_df')

     #next drop the duplicates and repeats from the frontier

     frontier_sdf=frontier_sdf.groupBy('Frontier').count().where('count=1')

     frontier_sdf.createOrReplaceTempView('frontier_df')

     #update the distance 
     distance = distance +1

     frontier_sdf=frontier_sdf.withColumn("distance",F.lit(distance))

     frontier_sdf.createOrReplaceTempView('frontier_df')

    #now union the visited with just the origin and the new frontier to get the new visited

     visited_sdf=spark.sql('''SELECT frontier_df.Frontier as node, frontier_df.distance FROM frontier_df UNION ALL SELECT visited_df.node, visited_df.distance FROM visited_df ''')
     #create the view for the visited sdf
     visited_sdf.createOrReplaceTempView('visited_df')


    #update the depths and distances
     depth=depth+1
     #distance=distance+1

  #grab the top 75 tuples and conver to pandas dataframe

  #find length of the spark df
  length_output=visited_sdf.count()

  answer_df=spark.sql('''SELECT * FROM  visited_df''')

  answer_df=answer_df.toPandas()

  answer_75 = answer_df.sort_values(by="node")[:75]

  print(length_output)


  return (answer_75)